# Efficient Two-Stream Network for Violence Detection Using Separable Convolutional LSTM
**Zahidul Islam, Mohammad Rukonuzzaman, Raiyan Ahmed, Md. Hasanul Kabir, Moshiur Farazi**

**Pembuat Kode Demo : Ahmad Wafiq Amrillah**


## Penjelasan Singkat
Penelitian ini menjelaskan tentang pengembangan sebuah metode yang efisien dan akurat untuk mendeteksi kekerasan secara otomatis dari rekaman video pengawasan. Ini merupakan bagian dari bidang pengenalan aktivitas manusia dalam Computer Vision.


## Prasyarat
Adapun library yang digunakan pada penelitian ini, antara lain:
* Python 3.7
* Tensorflow 2.5.0
* Scikit-image 0.16.2
* Scipy 1.4.1
* Pandas 1.1.5
* Matplotlib 3.2.2
* OpenCV 4.1.2.30
* Numpy 1.19.2
* Tqdm 4.41.1
* Pillow 8.1.2
* Scikit-learn 0.24.0

In [5]:
!python --version

Python 3.7.0


In [ ]:
%pip install -r requirements.txt

In [ ]:
import numpy as np
import cv2
import time
import IPython.display as display
from skimage import io 
from tensorflow.keras.models import load_model
from skimage import transform
from skimage import exposure
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt
from numpy import expand_dims
import argparse
import os
import io
import models
from dataGenerator import *
from datasetProcess import *

## Dataset
Dataset yang digunakan pada penelitian ini adalah :
* [RWF2000](https://github.com/mchengny/RWF2000-Video-Database-for-Violence-Detection). Dataset ini berisi video dari tiga jenis kekerasan, yaitu *violence*, *non-violence* dan *no-violence*. Namun, pada demo ini, dataset tidak dapat digunakan karena dataset tidak dapat di-download.
* [Hockey](https://www.kaggle.com/datasets/yassershrief/hockey-fight-vidoes) (../raw_videos/HockeyFights)
* [Movies](https://academictorrents.com/details/70e0794e2292fc051a13f05ea6f5b6c16f3d3635) (../raw_videos/movies)

Dataset yang diunduh disimpan dengan struktur folder seperti berikut : (Contoh untuk dataset Hockey)
```
📦project_directory
  ┣ 📂raw_videos
    ┣ 📂HockeyFights
```

## Training Model
Berikut proses pelatihan model yang dihasilkan pada penelitian ini.
Tetapi, sebelum melakukan pengujian model, diharapkan untuk melakukan proses training model pada dataset yang diinginkan dengan parameter ephocs yang sesuai dengan penelitian ini supaya dapat menghasilkan model yang akurat dan dataset memiliki content (npy) yang sesuai dengan penelitian ini.
***Untuk pelatihan model tidak dilakukan, sehingga menggunakan [model](https://github.com/zahid58/TwoStreamSepConvLSTM_ViolenceDetection) yang sudah di-train pada penelitian ini.***

In [ ]:
!python train.py --dataset hockey --vidLen 32 --batchSize 4 --numEpochs 150 --mode both --preprocessData --lstmType sepconv --savePath "./results"

## Evaluate Model
Berikut adalah contoh pengujian model yang dibuat pada penelitian ini.
***Untuk pengujian model menggunakan model yang sudah di-train, silahkan download model yang sudah di-train pada drive folder [trained_models](https://drive.google.com/drive/folders/1igx-plktW069IgXyWg3H78AKuTg-jCza?usp=sharing). Silahkan copy file model yang diinginkan ke folder project directory.***

In [ ]:
!python evaluate.py --dataset hockey --vidLen 32 --batchSize 4 --mode both --lstmType sepconv --fusionType M --weightsPath "./trained_models/hockey/sepconvlstm-M/model/hockey_model"

## Qualitative Analysis
Berikut adalah demonstrasi hasil model yang di-train secara visual dan memberikan pemahaman mendalam dari model yang di-train.

In [ ]:
def qualitative():
    weightsPath = "./trained_models/hockey/sepconvlstm-M/model/hockey_model"
    outputPath = "./Results/qualitativeAnalysis/"
    mode = "both"
    dataset = 'hockey'
    vid_len = 32
    dataset_frame_size = 320
    input_frame_size = 224
    frame_diff_interval = 1
    one_hot = False
    lstm_type = 'sepconv'
    
    preprocess_data = False
    if preprocess_data:
        if dataset == 'rwf2000':
            os.mkdir(os.path.join(dataset, 'processed'))
            convert_dataset_to_npy(
                src='{}/RWF-2000'.format(dataset),
                dest='{}/processed'.format(dataset),
                crop_x_y=None,
                target_frames=vid_len,
                frame_size= dataset_frame_size
            )
    
    test_generator = DataGenerator(
        directory='{}/processed/test'.format(dataset),
        batch_size=1,
        data_augmentation=False,
        shuffle=True,
        one_hot=one_hot,
        sample=False,
        resize=input_frame_size,
        target_frames = vid_len,
        frame_diff_interval = frame_diff_interval,
        dataset = dataset,
        normalize_ = False,
        background_suppress = False,
        mode = mode
    )
    
    print('> getting the model from...', weightsPath)  
    model =  models.getProposedModelM(size=224, seq_len=32, frame_diff_interval = 1, mode="both", lstm_type=lstm_type)
    model.load_weights(weightsPath).expect_partial()
    model.trainable = False
    model.summary()
    
    count = 100
    classes = {0:"violent", 1:"nonviolent"}
    tableResults = ""
    tableResults += "| index | target | predicted |\n"
    tableResults += "| ----- | ------ | --------- |\n"
    for i, (x,y) in enumerate(test_generator):
        if i == count:
            break
        data = x[0]; target = y[0]
        if i == 0:
            print(data.shape)
        data = np.squeeze(data)
        p = model.predict(x)
        p = np.squeeze(p)
        if p >= 0.50:
            predicted = 1
        else:
            predicted = 0   
        print("> index:",i, " target:",target, " predicted:",predicted)
        saveVideo(data, str(i)+"_GT-"+str(classes[target])+"_PL-"+str(classes[predicted]), outputPath, asFrames = True)
        tableResults += "| "+str(i)+" | "+str(classes[target])+" | "+str(classes[predicted])+" |\n"
        
    tableResults += "--------------------------------------------------------------------------------\n"
    print(tableResults)
    
def saveVideo(file, name, dest, asFrames = False, fps = 29, showImage = False):
    if file.dtype != np.uint8:    
        file = np.array(file, dtype = np.uint8)
    outpath = os.path.join(dest, name)
    _, h, w, _ = file.shape
    size = (h, w)
    if asFrames:
        os.mkdir(outpath)
        print("saving frames to ", outpath)
        print("number of frames:",len(file))
        for i in range(len(file)):
            filename = os.path.join(outpath, str(i)+".png")
            frame = cv2.cvtColor(file[i], cv2.COLOR_BGR2RGB)
            cv2.imwrite(filename, frame)
            if i == 0 and showImage:
                plt.imshow(frame)
                plt.axis('off')
                plt.show()
            
    else:
        fourcc = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X')
        print("saving video to ", outpath)
        out = cv2.VideoWriter(outpath,fourcc, fps, size)
        print("video length:",len(file))
        for i in range(len(file)):
            out.write(file[i])
        out.release()
    
qualitative()